In [1]:
import gc
import re
import time
import json
import pymongo
import numpy as np
import pandas as pd

In [ ]:
mongo_client = pymongo.MongoClient('dds-j6c504a39ac4fcc42259-pub.mongodb.rds.aliyuncs.com', 3717)
mongo_auth = mongo_client.monitoring
mongo_auth.authenticate('monitoringuser', 'passwordqwe')
db = mongo_client['monitoring']

In [ ]:
_all_blocks = db['all_heco_blocks']
_all_blocks = _all_blocks.find({"blockNumber":{"$lt":8589810}}).sort('blockNumber', -1).limit(25000)
#_all_blocks = _all_blocks.find({"blockNumber":{"$lt":8614810}}).sort('blockNumber', -1).limit(25000)
all_blocks = []
for block in _all_blocks:
    block.pop("_id")
    all_blocks.append(block)

In [6]:
with open("blocks_2.json", "w") as f:
    f.write(json.dumps(all_blocks))

In [7]:
with open("blocks_0.json") as f:
    all_blocks = json.load(f)

In [2]:
all_blocks = []
with open("blocks_1.json") as f:
    blocks = json.load(f)
all_blocks += blocks
del blocks
gc.collect()

0

In [2]:
all_blocks = []
with open("blocks_2.json") as f:
    blocks = json.load(f)
all_blocks += blocks
del blocks
gc.collect()

0

In [2]:
with open("pool_info.json", "r") as f:
    pools = json.load(f)
apools = {}
for symbol in pools:
    address = pools[symbol]["address"].lower()
    apools[address] = symbol

In [3]:
len(apools)

77

In [4]:
df = {}
n_total_transactions = 0
for block in all_blocks:
    transactions = block["transactions"]
    n_total_transactions += len(transactions)
    block_number = block["blockNumber"]
    block_timestamp = block["timestamp"]
    for transaction in transactions:
        if "Gas Price" not in transaction:
            continue
        else:
            gas_price = transaction["Gas Price"].replace(",", "").split("(")[-1].split(' ')[0]
            try:
                gas_price = float(gas_price)
            except:
                gas_price = np.nan

        tr_hash = transaction["Transaction Hash"]
        status = transaction["Status"]
        if "Success" in status:
            status = "Success"

        transferred_value = 0
        transferred_number = 0
        for token in transaction.get("Tokens", []):
            if not token.get("From", "") or token.get(
                    "To") == "0x0000000000000000000000000000000000000000 ":
                continue
            transferred_number += 1
            _ = token.get("For")
            if _:
                value = re.findall(r"[$](.*?)[)]", _)
                if len(value):
                    value = value[0]
                    value = value.replace(",", "")
                    try:
                        value = float(value)
                        transferred_value += float(value)
                    except:
                        print(_)

        if "Gas Limit" in transaction:
            gas_limit = transaction["Gas Limit"].replace(",", "")
            try:
                gas_limit = int(gas_limit)
            except:
                gas_limit = np.nan
        else:
            gas_limit = np.nan

        if "Gas Used by Transaction" in transaction:
            gas_used = transaction["Gas Used by Transaction"].split(' ')[0].replace(",", "")
            try:
                gas_used = int(gas_used)
            except:
                gas_used = np.nan
        else:
            gas_used = np.nan

        can_abi = transaction["canAbi"]

        logs = transaction.get("Logs", [])
        
        for log in logs:
            address = log["Address"]
            if address in apools:
                df[tr_hash] = {
                    "block_number": block_number,
                    "n_transaction": len(transactions),
                    "timestamp": block_timestamp,
                    "from": transaction.get("From"),
                    "to": transaction.get("To", "").replace("Contract", "").strip(),
                    "status": status,
                    "transferred_value": transferred_value,
                    "transferred_number": transferred_number,
                    "gas_limit": gas_limit,
                    "gas_used": gas_used,
                    "gas_price": gas_price,
                    "can_abi": can_abi,
                    "symbol": apools[address]
                }

df = pd.DataFrame.from_dict(df, orient="index")
df.index.name = "hash"
print(n_total_transactions)

924353


In [6]:
1197889+758081+924353

2880323

In [6]:
df.to_csv("cleaned.csv", mode="a", header=None)

In [2]:
df = pd.read_csv("cleaned.csv")

In [11]:
aa = df[df.gas_price==df.gas_price.max()]

In [10]:
df[df.block_number==8596486].iloc[0]

,hash,block_number,n_transaction,timestamp,from,to,status,transferred_value,transferred_number,gas_limit,gas_used,gas_price,can_abi,symbol
165030,0xf7240c6748925a590f9351fbcf9575cfd188d23682c8...,8596486,51,1.633744e+12,0x0000000000a3d531e72fe7bb40a4e7fb033595cd0x00...,0x466d229a514d7cdc0a331c450a1f4767217372770x46...,Success,1419.98,7,580000.0,313421,3618.452057,False,MDX/USDT
165031,0x4d93c77a2868052dd256e8e637cf098e463ef8aadeac...,8596486,51,1.633744e+12,0xaa3e34c024271c476a979fafc181aadb9d8ecffd0xaa...,0xed7d5f38c79115ca12fe6c0041abb22f0a06c3000xed...,Success,632.82,2,2100000.0,144550,41.000000,True,MDX/BCH
165032,0x290ae793cb02fa394a958dc9c889c3046c68f30898c0...,8596486,51,1.633744e+12,0x67108e315c95e345e02f741e95a904a62449b7c10x67...,0x2979f9c20f75891070f23f950700906551a560c60x29...,Success,678.85,3,380000.0,267348,11.999000,False,MDX/ETH
165033,0xd6d4e6c58ff92b5afd47c7f15ffad51096d5baf08dc4...,8596486,51,1.633744e+12,0xaccb69ec37083a13498ae673665c4dba4aef9a100xac...,0x92b3420a31250c62d019c530de4683676108a0950x92...,Success,1434.22,22,2860000.0,1057826,7.662765,False,MDX/USDT


In [16]:
len(set(df.block_number))

65055

In [3]:
df.status.value_counts()

Success    248811
Name: status, dtype: int64